In [1]:
from sklearn.model_selection import train_test_split
from glob import glob
import os
import shutil
import random
import pickle
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

In [11]:
def get_filepaths(directory,low,high,exten='.jpg'): 
    
    filepaths = []
    weird = [692, 693, 694, 695, 696, 697, 701, 705, 706, 707, 708, 709, 715, 718, 720, 698]
    for filename in os.listdir(directory):
        basename, ext = os.path.splitext(filename)
        if ext != exten:
            continue
        if int(basename) in weird:
            continue
        try:
            number = int(basename)
        except ValueError:
            continue  # not numeric
        if low <= number <= high:
            # process file
            filename = os.path.join(directory, filename)
            filepaths.append(filename)
            
    return filepaths

In [12]:
def copy_files(files,dst):
    
    old_files = glob(os.path.join(dst,'*'))
    for f in old_files:
        os.chmod(f,0o777)
        os.remove(f)
    for file in files:
        shutil.copy(file,dst)

In [13]:
img_src = 'E:/AI/FAPS/dataset/PA_Jakob_Roser_Datentrager/06_Datensatze/01_Images/Images_512x256'
seg_src = 'E:/AI/FAPS/dataset/PA_Jakob_Roser_Datentrager/06_Datensatze/01_Masks/Masks_JPG_512x256'

In [14]:
all_data_neg = get_filepaths(img_src,1,600)
all_data_pos = get_filepaths(img_src,601,1000)
all_data_seg = get_filepaths(seg_src,601,1000,'.png')

In [6]:
for split in [30,8]:
    
    train_pos_dst = f'E:/AI/FAPS/dataset/WS/{split}/train/pos'
    train_neg_dst = f'E:/AI/FAPS/dataset/WS/{split}/train/neg'
    train_seg_dst = f'E:/AI/FAPS/dataset/WS/{split}/train/seg'

    test_pos_dst = f'E:/AI/FAPS/dataset/WS/{split}/test/pos'
    test_neg_dst = f'E:/AI/FAPS/dataset/WS/{split}/test/neg'
    test_seg_dst = f'E:/AI/FAPS/dataset/WS/{split}/test/seg'
    
    train_pos_path,test_pos_path,train_seg_path,test_seg_path = train_test_split(all_data_pos,all_data_seg,random_state=1337)
    train_neg_path,test_neg_path = train_test_split(all_data_neg)
    random.seed(1337)
    train_seg_path = random.sample(train_seg_path,split)
#     copy_files(train_pos_path,train_pos_dst)
#     copy_files(test_pos_path,test_pos_dst)
#     copy_files(train_seg_path,train_seg_dst)
#     copy_files(test_seg_path,test_seg_dst)
#     copy_files(train_neg_path,train_neg_dst)
#     copy_files(test_neg_path,test_neg_dst)
    
    vals = [train_pos_path,test_pos_path,train_seg_path,test_seg_path,train_neg_path,test_neg_path]
    with open(f'E:\AI\FAPS\code\Mixedsupervision\splits\PA\split_{split}.pyb', "wb") as fp:   #Pickling
        pickle.dump(vals, fp)
    

In [32]:
def get_names(paths):
    names = []
    for p in paths:
        name = os.path.basename(os.path.normpath(p))
        name = int(name[:-4])
        names.append(name)
    return names

In [56]:
def read_split(num_seg):
    fn = f"PA/split_{num_seg}.pyb"
    with open(f"splits/{fn}", "rb") as f:
        vals = pickle.load(f)
    return vals

In [8]:
with open(f'labels/l_to_i.pyb', "rb") as fp:   #Pickling
    label_image_pair = pickle.load(fp)
with open(f'labels/i_l.pyb', "rb") as fp:   #Pickling
    image_label_pair = pickle.load(fp)

In [9]:
y_vals = []
for i in range(1,1001):
    y

array([691, 698, 699, 700, 702, 703, 704, 710, 711, 712, 713, 714, 716,
       717, 719, 891, 892, 901, 902, 911, 912, 913, 914, 915, 947, 948,
       953, 954, 959, 960, 961, 985, 989, 993, 994])

In [10]:
image_label_pair[691]

['3', '5']

In [41]:
seg = 0
train_pos_path,test_pos_path,train_seg_path,test_seg_path,train_neg_path,test_neg_path = read_split(seg)

In [51]:
paths = train_pos_path
img_names = get_names(paths)
count_dict = {}
for i in range(1,881):
    if i not in image_label_pair.keys():
        continue
    for l in image_label_pair[i]:
        if l in count_dict.keys():
            count_dict[l]+=1
        else:
            count_dict[l] = 1

In [121]:
X = np.array(list(image_label_pair.keys())).reshape(-1,1)
y = list(image_label_pair.values())

In [122]:
one_hot = MultiLabelBinarizer()

In [123]:
X_train, y_train, X_test, y_test = iterative_train_test_split(X, one_hot.fit_transform(y), test_size = 0.1)


In [1]:
from sklearn.datasets import make_multilabel_classification
X,Y = make_multilabel_classification(n_samples=100, n_classes=3, n_labels=2)

In [4]:
X

array([[0., 1., 3., ..., 0., 1., 0.],
       [0., 3., 4., ..., 0., 1., 0.],
       [0., 4., 3., ..., 4., 0., 2.],
       ...,
       [0., 2., 4., ..., 4., 2., 0.],
       [4., 3., 6., ..., 3., 1., 1.],
       [4., 1., 6., ..., 2., 1., 3.]])

In [133]:
count_dict = {}
for i in X_train:
    i = i[0]
    if i not in image_label_pair.keys():
        continue
    for l in image_label_pair[i]:
        if l in count_dict.keys():
            count_dict[l]+=1
        else:
            count_dict[l] = 1
count_dict

{'3': 751,
 '9': 45,
 '2': 45,
 '10': 45,
 '12': 44,
 '11': 45,
 '8': 45,
 '7': 45,
 '6': 43,
 '1': 45,
 '5': 31,
 '13': 44,
 '4': 45}

In [132]:
X_train.shape

(886, 1)

In [128]:
X.shape

(985, 1)

In [129]:
test = X_train.reshape(-1)

In [130]:
len(np.unique(test))

886

In [144]:
image_label_pair[844]

['3', '12']

In [8]:
x = np.arange(20).reshape(2,2,5)
y = np.where(x < 3 , -1,x)
y

array([[[-1, -1, -1,  3,  4],
        [ 5,  6,  7,  8,  9]],

       [[10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]]])

In [10]:
np.where(y == -1 , np.array([0,0,0]),np.array([2,3,3]) )

ValueError: operands could not be broadcast together with shapes (2,2,5) (3,) (3,) 